In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# --- Configuration ---
START_DATE = "2021-07-31"
END_DATE = "2025-07-31"
BACKTEST_START = "2024-01-01"
TICKER = "ETH-USD"

# Strategy Parameters
LEAD = 20
LAG = 55
THRESHOLD = 0.005  # 0.5%
DRAWDOWN_STOP = 0.20  # 20% max drawdown


# --- Download Data ---
def download_data():
    df = yf.download(TICKER, start=START_DATE, end=END_DATE)
    df = df[['Close']].dropna()
    df.index = pd.to_datetime(df.index)
    return df


# --- Signal Generation ---
def generate_signals(df, lead, lag, threshold):
    df = df.copy()
    df['SMA_lead'] = df['Close'].rolling(window=lead).mean()
    df['SMA_lag'] = df['Close'].rolling(window=lag).mean()
    df['diff'] = (df['SMA_lead'] - df['SMA_lag']) / df['SMA_lag']
    df['Signal'] = 0
    df.loc[df['diff'] > threshold, 'Signal'] = 1
    df.loc[df['diff'] < -threshold, 'Signal'] = -1
    return df


# --- Strategy A: SMA only ---
def apply_simple_sma_strategy(df):
    df = df.copy()
    df['Return'] = df['Close'].pct_change()
    df['Position'] = df['Signal'].shift()
    df['Strategy'] = df['Position'] * df['Return']
    df['StrategyCumulative'] = (1 + df['Strategy']).cumprod()
    return df


# --- Strategy B: SMA + Drawdown + Re-entry ---
def apply_drawdown_stoploss_with_reentry(df, max_dd_threshold=0.2):
    df = df.copy()
    df['Return'] = df['Close'].pct_change()
    df['Position'] = 0.0

    strategy_value = 1.0
    peak_value = 1.0
    drawdown_active = False

    df_reset = df.reset_index(drop=True)

    for i in range(1, len(df_reset)):
        signal = df_reset['Signal'].iloc[i - 1]
        daily_ret = df_reset['Return'].iloc[i]
        position = df_reset['Position'].iloc[i - 1]

        if drawdown_active:
            if signal != 0:
                position = signal
                drawdown_active = False
        else:
            if signal != 0:
                position = signal

        strategy_value *= (1 + position * daily_ret)
        peak_value = max(peak_value, strategy_value)

        if (strategy_value - peak_value) / peak_value <= -max_dd_threshold:
            drawdown_active = True
            position = 0

        df_reset.iloc[i, df_reset.columns.get_loc('Position')] = position

    df_reset['Strategy'] = df_reset['Position'] * df_reset['Return']
    df_reset['StrategyCumulative'] = (1 + df_reset['Strategy']).cumprod()
    df_reset['Cumulative'] = (1 + df_reset['Return']).cumprod()
    df_reset.index = df.index

    return df_reset


# --- Evaluate Performance ---
def compute_max_drawdown(series):
    roll_max = series.cummax()
    drawdown = (series - roll_max) / roll_max
    return drawdown.min()


def evaluate(series):
    ret = series.pct_change().dropna()
    ann_return = ret.mean() * 252
    ann_vol = ret.std() * np.sqrt(252)
    sharpe = ann_return / ann_vol if ann_vol != 0 else 0
    total_return = series.iloc[-1] - 1
    max_dd = compute_max_drawdown(series)
    return {
        "Annualized Return (%)": round(ann_return * 100, 2),
        "Annualized Volatility (%)": round(ann_vol * 100, 2),
        "Sharpe Ratio": round(sharpe, 2),
        "Total Return (%)": round(total_return * 100, 2),
        "Max Drawdown (%)": round(max_dd * 100, 2)
    }


# --- Plot Results Together ---
def plot_all_strategies(df_base, df_sma_only, df_dd, title):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df_dd.index, y=df_base['Cumulative'],
                             name="Buy & Hold", line=dict(width=2, color='gray')))
    fig.add_trace(go.Scatter(x=df_dd.index, y=df_sma_only['StrategyCumulative'],
                             name="SMA Only", line=dict(dash='dot', width=2, color='orange')))
    fig.add_trace(go.Scatter(x=df_dd.index, y=df_dd['StrategyCumulative'],
                             name="SMA + MaxDD + Re-entry", line=dict(dash='dash', width=2, color='cyan')))

    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title='Cumulative Return',
        template='plotly_dark',
        legend=dict(x=0, y=1.05, orientation="h")
    )
    fig.show()


# --- Main ---
def main():
    df_all = download_data()
    df_bt = df_all.loc[BACKTEST_START:]

    print(f"Running backtest: Lead={LEAD}, Lag={LAG}, Threshold={THRESHOLD}, MaxDD={DRAWDOWN_STOP * 100:.0f}%")

    df_bt = generate_signals(df_bt, LEAD, LAG, THRESHOLD)

    df_simple = apply_simple_sma_strategy(df_bt)
    df_full = apply_drawdown_stoploss_with_reentry(df_bt, DRAWDOWN_STOP)
    df_base = df_simple.copy()
    df_base['Cumulative'] = (1 + df_base['Return']).cumprod()

    # Evaluate all strategies
    print("\n--- Performance Metrics ---")
    metrics_bh = evaluate(df_base['Cumulative'])
    metrics_sma = evaluate(df_simple['StrategyCumulative'])
    metrics_dd = evaluate(df_full['StrategyCumulative'])

    print("\n[Buy & Hold]")
    for k, v in metrics_bh.items():
        print(f"{k}: {v}")

    print("\n[SMA Only]")
    for k, v in metrics_sma.items():
        print(f"{k}: {v}")

    print("\n[SMA + MaxDD + Re-entry]")
    for k, v in metrics_dd.items():
        print(f"{k}: {v}")

    # Plot
    plot_all_strategies(
        df_base,
        df_simple,
        df_full,
        f"ETH Strategy Comparison | SMA={LEAD}/{LAG}, Threshold={THRESHOLD}, MaxDD={int(DRAWDOWN_STOP * 100)}%"
    )


if __name__ == "__main__":
    main()


/tmp/ipython-input-2106672609.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(TICKER, start=START_DATE, end=END_DATE)
[*********************100%***********************]  1 of 1 completed


Running backtest: Lead=20, Lag=55, Threshold=0.005, MaxDD=20%

--- Performance Metrics ---

[Buy & Hold]
Annualized Return (%): 37.9
Annualized Volatility (%): 58.51
Sharpe Ratio: 0.65
Total Return (%): 61.89
Max Drawdown (%): -63.79

[SMA Only]
Annualized Return (%): 32.04
Annualized Volatility (%): 55.74
Sharpe Ratio: 0.57
Total Return (%): 44.95
Max Drawdown (%): -49.92

[SMA + MaxDD + Re-entry]
Annualized Return (%): 53.14
Annualized Volatility (%): 33.72
Sharpe Ratio: 1.58
Total Return (%): 195.39
Max Drawdown (%): -19.77
